# Let's Dive into Pandas World

Here's what a typical data science project looks like:
1. Problem identification
2. **EDA**
3. **Data Cleaning**
4. **Feature Engineering**
5. Modeling
6. Project Delivery 

During this class we will discuss and work on 2, 3 and 4 points from the list above with Pandas.

### Exploratory Data Analysis:

*“Exploratory data analysis” is an attitude, a state of flexibility, a willingness to look for those things that we believe are not there, as well as those we believe to be there.*
<br>
— John Tukey
<br> 
<br>
During EDA, we use plots, graphs, and summary statistics to gain more understanding and intuition about the dataset. 
<br>
<br>
Generally speaking, it’s a method of systematically going through the data, plotting distributions of all variables (using box plots), plotting time series of data, transforming variables, looking at all pairwise relationships between variables using scatterplot matrices, and generating summary statistics for all of them. [1]
<br>
<br>
<br>
**A simple guide to help you conduct EDA:**
1. Dataset overview
    - Exploring summary statistics of variables
    - Exploring NAs
2. Going deeper by examining each variable individually
    - Visualisation of each variable 
    - Outlier detection
3. Exploring the interaction of two or more variables
    - Visualisation techniques
    - Correlation matrices 

<br>

**How to clean up a dataset:**  
1. Dealing with unuseful data
2. Dealing with NAs
3. Dealing with outliers
4. Dealing with Duplicates

<br>

**Creating new features with Feature Engineering:**
1. Many new variables from datetime objects
2. Changing data type (e.g. continuous to categorical)
3. Creating dummy variables
4. Creating new variables using the sum, product, difference of two or more variables
5. Using your imagination for more 

<br>
<br>
Further you can find application to the methods and techniques described above. This is a general EDA and preprocessing analysis that can be used later for modeling. 

[Link to dataset](https://www.kaggle.com/carrie1/ecommerce-data):


"This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers."


In [ ]:
# Packages
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import date
%matplotlib inline

In [ ]:
filename = 'data.csv'
folder = 'data'
path = os.path.join(folder, filename)
# path = 'data/InvoiceData.csv'
data = pd.read_csv(path, encoding='unicode-escape')

In [ ]:
# Get the Top and Buttom n elements
# data.head() 
# data.tail(5)

In [ ]:
# Get Info About Data types
data.info()

In [ ]:
# Get the main statistics of your data
data.describe()

In [ ]:
print(f'Checking missig values\n {data.isna().sum()}')

In [ ]:
print(f'Checking for duplicates values\n {data.duplicated().sum()}')

### Exploratory Data Analysis

##### <div align="center">  Metadata </div>        

| **Column Name** |     **Description**  |  
|----------       |:-------------:       |
| InvoiceNo       |  Invoice Number      |
| StockCode       |  Product code        |
| Description     |  Product description |
| Quantity        |  Number of purchased items |
| InvoiceDate     |  Invoice Date        |
| UnitPrice       |  Price for one item  |
| CustomerID      |  Customer ID         |
| Country         |  Customer's country  |

In [ ]:
# General information about dataset
# print(type(data.CustomerID.values))
data.info()

In [ ]:
# Change data types
# data.InvoiceDate == data['InvoiceDate']
data['InoviceDate'] = pd.to_datetime(data['InvoiceDate'])
data['CustomerID'] = data['CustomerID'].astype(str)# astype('O'|str|int|float...)
# data.InvoiceNo.apply(lambda x: str(x))

In [ ]:
data.select_dtypes(exclude='O') # include='O'

In [ ]:
# Data Slicing example
focus_columns = ['Quantity', 'InvoiceDate']
data[focus_columns][:2000]

# data.loc[:2000, 'Quantity']
# data.iloc[:2000, 3]

In [ ]:
# How to create a DataFrame from Scratch
DD = {"A": [1,2,3,4,5], "N": [8,9,10,11,12]}
pd.DataFrame(DD).to_dict(orient='list')

### 1.Dealing with NAs

In [ ]:
# Number of NAs and Percentage of NAs in dataset
# data.shape -> (i, j)
data_nas = pd.DataFrame(data.isna().sum(), columns=['Number of NAs'])
data_nas['Percentage of NAs'] = round(data_nas['Number of NAs'] / data.shape[0] * 100, 3)
data_nas

The main goal of this EDA and future analysis is to understand the behavior of company customers, which is why we need to remove NAs in 'CustomerID' column. Moreover, by removing these values from 'Customer ID' we clean 'Description' NAs as well.

In [ ]:
# Instead we can drop columns CustomerID and Description
# data = data[['InvoiceNo', 'StockCode', 'Quantity', 'InvoiceDate',
#        'UnitPrice', 'Country']]
data.dropna(subset=['CustomerID'], inplace=True)

In [ ]:
data.shape

In [ ]:
data.isna().sum()

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
# To Be Continued

### 2. Exploring each variable separately

In [ ]:
print("Numbers of observations: ", data.shape[0])
# print("Numbers of unique invoices: ", data.InvoiceNo.nunique())
# print('Number of unique customers: ', data.CustomerID.nunique())
print('Numbere of unique products:', data.StockCode.nunique())

In [ ]:
for i in data.columns:
    print(f"Column name:{i}",
          "\nUnique values: ", data[i].unique(),
          "\nNumber of unique values: ", data[i].nunique(), "\n____")

Here we can see the huge difference between number of observations and numbers of unique orders, which means that database has no order level structure. 

##### Continuous data visualisation

In [ ]:
plt.figure(figsize=(10, 5))
sns.distplot(data.Quantity)
plt.axvline(data.Quantity.mean(), color='k', linestyle='dashed', linewidth=1)
plt.axvline(data.Quantity.median(), color='r', linewidth=1)
plt.title('Distribution of variable "Quantity"')
plt.xlabel("Quantity")
# plt.show()

In [ ]:
plt.figure(figsize=(10, 5)) 
sns.distplot(data.UnitPrice)
plt.axvline(data.UnitPrice.mean(), color='k', linestyle='dashed', linewidth=1)
plt.axvline(data.UnitPrice.median(), color='r', linewidth=1)
plt.title('Distribution of variable "UnitPrice"')
plt.xlabel("UnitPrice")
plt.show()

In [ ]:
%%time
plt.figure(figsize=(10,5))
sns.lineplot(x=data.InvoiceDate[:10000], y=data.Quantity[:10000]) # Boxplot
plt.xticks(rotation = 70)
plt.show()

Both of the above graphs show that we have outliers for both variables. We will explore them more deeply.

In [ ]:
# Datetime variable
data['InvoiceDate'].describe()

In analyzing customer behavior, date and time variables will play a major role in solving various types of problems. This will be useful later in the analysis of retention, loyalty, segmentation, etc. 

##### Categorical data visualisation

In [ ]:
# With continuous - Histpgram , with categorical -  Barplot 
plt.figure(figsize=(10,5))
sns.countplot(data.Country) # Barplot 
plt.title('Frequency of variable "Country"')
plt.xlabel("Country")
plt.ylabel("Frequency in each class")
plt.xticks(rotation = 70)
plt.show()

Most customers are based in UK. From the point of view of the 'Country' variable, the data is highly imbalanced.
<br>
<br>
There is no need to visualize other categorical variables, because these variables have many unique values and we can't gain any valuable information from it.

### 3. Outlier detection

In [ ]:
data.Quantity.describe()

In [ ]:
data.UnitPrice.describe()

In [ ]:
# Exploring large values  of 'UnitPrice' variable
larges = data[data['UnitPrice']>=500] # .loc but it's not that beautiful 
sns.scatterplot(data=larges,x='UnitPrice', y='Quantity')
plt.title('Scatterplot of "Unit Price" and "Quantity"')
plt.xlabel("Unit Price")
plt.ylabel("Quantity")
plt.show()

In [ ]:
#Let's look at categorical variables of this part of the data once again
larges.describe(include = 'object')

In [ ]:
# Unique values of 'StockCode' variable.
larges.StockCode.unique()

In [ ]:
# Unique values of 'Description' variable.
# larges.Description.unique()

**Some assumptions:**
1. From the plot above and from the description of the 'Quantity' variable, it is clear that we have a lot of data issues. First of all, the quantity variable cannot take a negative value. Moreover, our maximum value is very large. It is doubtful that a customer can buy the same product so many times. 
<br>
<br>
2. Next variable with outliers is 'UnitPrice'. The only problem with this is the very large values.To understand the reasons for their occurrence, we only consider observations at unit prices of 500 or more. Some interesting relations can be found:
    <br>
    2.1 High unit prices mostly have negative values for the Quantity variable. 
    <br>
    2.2 The variables 'StockCode 'and' Description 'show that these observations are basically not ordinary products.  
<br>
<br>

**What to do next:**
1. Remove negative and very large values from 'Quantity' variable.
2. Remove StockCode and Description values which do not apply to usual products.

In [ ]:
# Remove negative values from 'Quantity' variable and very large values of 'Quantity' variable
filtered = data[data['Quantity'] >= 0]
filtered = filtered[filtered['Quantity']<5000]

In [ ]:
# Remove not real product codes from StockCode and very large values of 'Unit Price' variable
vals = ['M', 'D', 'POST',  'DOT', 'CRUK']
filtered = filtered[~filtered['StockCode'].isin(vals)] # Syntactic Suger sample

filtered = filtered[filtered['UnitPrice']<500]

In [ ]:
sns.distplot(filtered.Quantity)

In [ ]:
# TEST IT IN REGRESSION
# sparse vector proble 
weekday = filtered.InvoiceDate.dt.weekday.astype('category')
country = filtered.Country.astype('category')

### 4. Feature engineering

Creating new variables will help to find more relations in data. They may be used in later analysis as well.

In [ ]:
filtered['TotalPrice'] = filtered['Quantity'] * filtered['UnitPrice']
filtered['OrderHour'] = filtered.InvoiceDate.dt.hour
filtered['OrderMonth'] = filtered.InvoiceDate.dt.month
filtered['OrderWeekday'] = filtered.InvoiceDate.dt.weekday
filtered

In [ ]:
filtered.to_csv("data/data_cleared.csv")

### 5. Exploring interaction of two or more variables

In [ ]:
num_cols = ['Quantity', "UnitPrice", 'TotalPrice', 'OrderHour', 'OrderMonth']
# It may take long time to process
# sns.pairplot(filtered, vars = num_cols)

In [ ]:
corr = filtered.select_dtypes(exclude="O").corr()

plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
ax = sns.heatmap(corr, vmin=-1, vmax=1, center=0, annot=corr)

The only relationship we can find is between Quantity and Total price, which has an obvious reason: TotalPrice was created from multiplying Quantity and Unit prices.

In [ ]:
sns.barplot(x=filtered["OrderWeekday"], y=filtered["Quantity"] )
plt.title('Barplot of "OrderWeekday" and "Quantity" variables')
plt.xlabel("Variable 'OrderWeekday'")
plt.ylabel("Variable 'Quantity'")
plt.xticks(rotation=50)
plt.show()

In [ ]:
sns.barplot(x=filtered["OrderMonth"],y=filtered["Quantity"] )
plt.title('Barplot of "OrderMonth" and "Quantity" variables')
plt.xlabel("Variable 'OrderMonth'")
plt.ylabel("Variable 'Quantity'")
plt.xticks(rotation=50)
plt.show()

In [ ]:
sns.barplot(x=filtered["OrderHour"], y=filtered["Quantity"] )
plt.title('Barplot of "OrderHour" and "Quantity" variables')
plt.xlabel("Variable 'OrderHour'")
plt.ylabel("Variable 'Quantity'")
plt.xticks(rotation=50)
plt.show()

The relation between time variables and 'Quantity' variable:

- Minimum order quantity reached on Sunday.
- November has the least quantity of ordered products. 
- Customers usually make purchases at 7am.

**Conclusion**

This is just an example of EDA and data preprocessing that you can conduct. You can dig deeper, find more relationships, add more data and everything you can think of. This is an art and you're the master!

More importantly, this is just the beginning. We will use this data, knowledge and intuition, which we obtained as a result of this analysis, to go further and solve many problems related to customer behavior.

**Further Reading**
<br>
[1] [Doing Data Science](https://www.oreilly.com/library/view/doing-data-science/9781449363871/)
<br>
[2] [R for Marketing Research and Analytics](https://www.springer.com/gp/book/9783030143152)